In [26]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'Century'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [27]:
@njit()
def valueOf(card, resource):
    score = card[4]
    cost = card[0:4]
    # needs = cost - resource
    # needs[needs<=0] = 0
    return score/(np.sum(cost))

@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions = np.where(validActions == 1)[0]

    resources = state[2:6]
    cards = state[194:219].reshape(5,-1)
    purchasePointCardActions = validActions[(validActions>=7) & (validActions<12)]
    if len(purchasePointCardActions) > 0:
        valueOfCards = np.zeros_like(purchasePointCardActions) - 1
        for i in range(len(purchasePointCardActions)):
            valueOfCards[i] = valueOf(cards[int(purchasePointCardActions[i]-7)], resources)
        action = purchasePointCardActions[0]
        for i in range(len(purchasePointCardActions)):
            if valueOfCards[i] > valueOf(cards[int(purchasePointCardActions[i]-7)], resources):
                action = purchasePointCardActions[i]
        return action, per 
    
    purchaseActionCardActions = validActions[(validActions>=1) & (validActions<7)]
    if len(purchaseActionCardActions) > 0:
        return purchaseActionCardActions[0], per
    
    performActionCardsActions = validActions[(validActions>=12) & (validActions<57)]
    if len(performActionCardsActions) > 0:
        return performActionCardsActions[0], per
    
    returnTokenActions = validActions[(validActions>=57) & (validActions<61)]
    if len(returnTokenActions) > 0:
        return returnTokenActions[0], per
    
    upgradeTokenActions = validActions[(validActions>=62) & (validActions<65)]
    if len(upgradeTokenActions) > 0:
        return upgradeTokenActions[0], per
    
    if 61 in validActions:
        return 61, per 
    
    if 0 in validActions:
        return 0, per
    
    action = validActions[np.random.randint(len(validActions))]
    return action, per


In [29]:
win, per = numba_main_2(player, 10000, np.array([]), 1)
win

2588